In [50]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [51]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 25
batch_size = 100
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified


In [57]:
torch.cuda.empty_cache()

![resnetimage](https://user-images.githubusercontent.com/30661597/78585170-f4ac7c80-786b-11ea-8b00-8b751c65f5ca.PNG)

In [58]:
def conv3x3(in_channels, out_channels, stride = 1):
    return nn.Conv2d(in_channels, out_channels, kernel_size= 3, stride= stride, padding= 1, bias= False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsampling = None):
        super().__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace= True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsampling = downsampling
        
    def forward(self, x):
        residual = x.clone()
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsampling:
            residual = self.downsampling(x)
        out += residual
        out = self.relu(out)
        return out

In [59]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super().__init__()
        self.in_channels = 16
        self.conv = conv3x3(3,16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace= True)
        self.layer1 = self.make_layer(block, 16, layers[0], stride = 1)
        self.layer2 = self.make_layer(block, 32, layers[1], stride = 2)
        self.layer3 = self.make_layer(block, 64, layers[2], stride = 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, layer, stride = 1):
        downsampling = None
        if(self.in_channels != out_channels) or (stride != 1):
            downsampling = nn.Sequential(conv3x3(self.in_channels, out_channels, stride),
                                         nn.BatchNorm2d(out_channels))
        
        residual_blocks = []
        residual_blocks.append(block(self.in_channels, out_channels, stride = stride, downsampling = downsampling))
        self.in_channels = out_channels
        for i in range(1, layer):
            residual_blocks.append(block(self.in_channels, out_channels))
        return nn.Sequential(*residual_blocks)
    
    def forward(self, x):
        print(x.size())
        out = self.conv(x)
        print(out.size())
        out = self.bn(out)
        print(out.size())
        out = self.relu(out)
        print(out.size())        
        out = self.layer1(out)
        print(out.size())
        out = self.layer2(out)
        print(out.size())
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0),-1) #spłaszczanie
        out = self.fc(out)
        return out

In [60]:
model = ResNet(ResidualBlock, [2,2,2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 2.00 GiB total capacity; 234.00 MiB already allocated; 0 bytes free; 392.00 MiB reserved in total by PyTorch)

In [45]:
decay = 0
model.train()
for epoch in range(num_epochs):
    
    # decay the learning rate by a factor of 0.5 every 20 epochs:
    if (epoch + 1)% 20 == 0:
        decay += 1
        optimizer.param_groups[0]['lr'] *= 0.5
        #optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print("New learning rate is: ", optimizer.param_groups[0]['lr'])
    
    for i, (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        print(images.shape)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 16, 32, 32])
torch.Size([100, 16, 32, 32])
torch.Size([100, 16, 32, 32])
torch.Size([100, 16, 32, 32])


RuntimeError: Given groups=1, weight of size [32, 16, 3, 3], expected input[100, 32, 16, 16] to have 16 channels, but got 32 channels instead

In [ ]:
#Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))